In [1]:
import pandas as pd

Import all the files

In [2]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [3]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [4]:
visits_cart = pd.merge(visits, cart, how="left")
visits_cart.head()

,user_id,visit_time,cart_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT


Step 3: How long is `visits_cart`?

In [5]:
total_visits = len(visits_cart)
print(total_visits)

2000


Step 4: How many timestamps are null for `cart_time`?

In [6]:
null_cart = visits_cart.cart_time.isnull().sum()
print(null_cart)

1652


Step 5: What percentage only visited?

In [7]:
visit_only_percentage = null_cart/total_visits * 100
print(visit_only_percentage)

82.6


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [8]:
cart_checkout = pd.merge(cart, checkout, how="left")

made_cart =len(cart)
no_checkout = cart_checkout.checkout_time.isnull().sum()
no_checkout_percentage = no_checkout / len(cart) * 100
print(f"Cart but not checkout: {no_checkout_percentage}")

Cart but not checkout: 35.05747126436782


Step 7: Merge it all together

In [9]:
complete_merge = visits_cart.merge(cart_checkout, how="left").merge(purchase, how="left")
complete_merge.head()


,user_id,visit_time,cart_time,checkout_time,purchase_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT


Step 8: % of users who got to checkout but did not purchase

In [10]:
reached_checkout = complete_merge[complete_merge.checkout_time.notnull()]
no_purchase = complete_merge[(complete_merge.purchase_time.isnull()) & (complete_merge.checkout_time.notnull())]
no_purchase_percentage = len(no_purchase) / len(reached_checkout) * 100
print(f"% of users who got to checkout but did not purchase: {no_purchase_percentage}")

% of users who got to checkout but did not purchase: 24.550898203592812


Step 9: check each part of the funnel, let's print all 3 of them again

In [11]:
print(f"Number of visits: {visit_only_percentage}")
print(f"Cart but not checkout: {no_checkout_percentage}")
print(f"% of users who got to checkout but did not purchase: {no_purchase_percentage}")

Number of visits: 82.6
Cart but not checkout: 35.05747126436782
% of users who got to checkout but did not purchase: 24.550898203592812


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [12]:
complete_merge["visit_to_purchase"] = complete_merge.apply(lambda row: row["purchase_time"] - row["visit_time"]\
                                                           if row["purchase_time"] is not None and row["visit_time"] is not None\
                                                           else None, axis=1)

complete_merge.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time,visit_to_purchase
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00,0 days 00:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT,NaT


In [13]:
complete_merge['time_to_purchase'] = \
    complete_merge.purchase_time - \
    complete_merge.visit_time
complete_merge.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time,visit_to_purchase,time_to_purchase
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00,0 days 00:44:00,0 days 00:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT,NaT,NaT


Step 12: average time to purchase

Step 11: examine the results

In [14]:
complete_merge.describe()

,visit_time,cart_time,checkout_time,purchase_time,visit_to_purchase,time_to_purchase
count,2108,456,334,252,252,252
mean,2017-06-16 23:26:43.036052992,2017-06-14 00:56:16.052631552,2017-06-13 05:18:17.425149440,2017-06-12 07:04:02.380952320,0 days 00:43:12.380952380,0 days 00:43:12.380952380
min,2017-01-01 03:47:00,2017-01-01 03:58:00,2017-01-04 05:32:00,2017-01-05 02:45:00,0 days 00:09:00,0 days 00:09:00
25%,2017-03-21 07:06:00,2017-03-19 00:10:00,2017-03-21 15:29:00,2017-03-22 08:25:30,0 days 00:32:00,0 days 00:32:00
50%,2017-06-19 00:01:30,2017-06-08 17:58:00,2017-06-04 20:40:00,2017-05-26 22:25:00,0 days 00:44:30,0 days 00:44:30
75%,2017-09-12 07:47:15,2017-09-06 04:27:30,2017-09-05 04:33:15,2017-09-04 17:39:00,0 days 00:54:00,0 days 00:54:00
max,2017-11-27 09:40:00,2017-11-27 08:54:00,2017-11-27 09:18:00,2017-11-24 05:38:00,0 days 01:17:00,0 days 01:17:00
std,NaN,NaN,NaN,NaN,0 days 00:14:42.168126436,0 days 00:14:42.168126436


In [15]:
complete_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2108 entries, 0 to 2107
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   user_id            2108 non-null   object         
 1   visit_time         2108 non-null   datetime64[ns] 
 2   cart_time          456 non-null    datetime64[ns] 
 3   checkout_time      334 non-null    datetime64[ns] 
 4   purchase_time      252 non-null    datetime64[ns] 
 5   visit_to_purchase  252 non-null    timedelta64[ns]
 6   time_to_purchase   252 non-null    timedelta64[ns]
dtypes: datetime64[ns](4), object(1), timedelta64[ns](2)
memory usage: 115.4+ KB


In [16]:
complete_merge.head(20)

,user_id,visit_time,cart_time,checkout_time,purchase_time,visit_to_purchase,time_to_purchase
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00,0 days 00:44:00,0 days 00:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT,NaT,NaT
5,7bc62113-d51d-4e6b-85e0-1cf29e3de74a,2017-06-03 20:05:00,NaT,NaT,NaT,NaT,NaT
6,6f22cd44-bc63-4449-a9af-2651859da2f9,2017-03-18 23:09:00,NaT,NaT,NaT,NaT,NaT
7,49a6399b-c095-4e42-92eb-af90abe2bb6a,2017-01-10 12:09:00,NaT,NaT,NaT,NaT,NaT
8,653c095d-fd74-40dd-ae3b-5a69a2613629,2017-06-27 19:21:00,NaT,NaT,NaT,NaT,NaT
9,3d9fdac1-aae3-4a01-b448-934d12047468,2017-09-24 06:43:00,NaT,NaT,NaT,NaT,NaT


In [17]:
mean_time = complete_merge.visit_to_purchase.mean()
print(mean_time)

0 days 00:43:12.380952380
